In [42]:
import torch
import mano
from mano.utils import Mesh

model_path = './_DATA/data/mano'
n_comps = 45
batch_size = 10

rh_model = mano.load(model_path=model_path,
                     is_rhand= True,
                     num_pca_comps=n_comps,
                     batch_size=batch_size,
                     flat_hand_mean=False)

betas = torch.rand(batch_size, 10)*.1
pose = torch.rand(batch_size, n_comps)*.1
global_orient = torch.rand(batch_size, 3)
transl        = torch.rand(batch_size, 3)

output = rh_model(betas=betas,
                  global_orient=global_orient,
                  hand_pose=pose,
                  transl=transl,
                  return_verts=True,
                  return_tips = True)


h_meshes = rh_model.hand_meshes(output)
j_meshes = rh_model.joint_meshes(output)

In [43]:
import numpy as np
import plotly.graph_objects as go

def display_mesh_plotly(mesh):
    """
    mano.utils.Mesh 객체의 vertices와 faces를 Plotly의 Mesh3d로 변환하여,
    Notebook 내에서 인터랙티브하게 시각화합니다.
    """
    # mesh가 vertices와 faces 속성을 가지고 있다고 가정합니다.
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    # faces 배열의 각 행은 삼각형의 세 꼭짓점 인덱스를 나타냅니다.
    i = faces[:, 0]
    j = faces[:, 1]
    k = faces[:, 2]
    
    fig = go.Figure(data=[
        go.Mesh3d(
            x=x, y=y, z=z,
            i=i, j=j, k=k,
            opacity=0.50,
            color='lightblue',
            flatshading=True
        )
    ])
    fig.update_layout(
        title="3D Mesh Visualization with Perspective",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')  # 카메라를 perspective 투영으로 설정.
            )
        )
    )
    fig.show()

# 사용 예시:
display_mesh_plotly(h_meshes[0])

In [45]:
def display_hand_mesh_with_skeleton(hand_mesh, core_joints):
    """
    손 메쉬와 함께 핵심 손 스켈레톤 (21개 key joint) 점들을 overlay 시각화합니다.
    
    인자:
        hand_mesh: mano hand mesh 객체. vertices와 faces 속성이 있어야 합니다.
        core_joints: (21, 3) shape의 핵심 관절 좌표.
                     보통 output.joints[0] 와 같이 사용됩니다.
    """
    import numpy as np
    import plotly.graph_objects as go
    
    # 손 메쉬 데이터 추출
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    i = faces[:, 0]
    j_vals = faces[:, 1]
    k = faces[:, 2]
    
    # Plotly figure 생성 및 손 메쉬 추가 (투시 렌더링)
    fig = go.Figure()
    fig.add_trace(go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j_vals, k=k,
        opacity=0.3,          # 내부가 보이도록 불투명도 낮춤
        color='lightblue',
        flatshading=True,
        name="Hand Mesh"
    ))
    
    # 핵심 관절 좌표 (21개) 오버레이
    core_joints = np.array(core_joints)
    # 21개 관절에 사용할 색상 리스트 (원하는 색상으로 수정 가능)
    colors = [
        'red', 'green', 'blue', 'orange', 'purple',
        'cyan', 'magenta', 'yellow', 'brown', 'pink',
        'gray', 'olive', 'lime', 'maroon', 'navy',
        'teal', 'silver', 'gold', 'black', 'indigo', 'coral'
    ]
    
    for idx in range(core_joints.shape[0]):
        fig.add_trace(go.Scatter3d(
            x=[core_joints[idx, 0]],
            y=[core_joints[idx, 1]],
            z=[core_joints[idx, 2]],
            mode='markers',
            marker=dict(size=6, color=colors[idx % len(colors)]),
            name=f'Joint {idx}'
        ))
    
    # 카메라 투시 설정
    fig.update_layout(
        title="Hand Mesh with Core Skeleton Points",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')  # 투시 (perspective) 방식
            )
        )
    )
    
    fig.show()


# 사용 예시:
# MANO 모델 출력의 첫번째 샘플에 대한 손 메쉬와, 핵심 관절 21개 점을 함께 시각화합니다.
display_hand_mesh_with_skeleton(h_meshes[0], output.joints[0])

In [40]:
def display_skeleton_with_local_axes(joints, hand_mesh, axis_length=0.05):
    """
    각 관절의 위치에서, 해당 관절에 고유한 로컬 좌표계(3축)를 시각화합니다.
    
    각 관절에 대해:
      - y축: 관절의 뼈(bone) 방향. 
             (자식이 존재하면, 자식 방향의 평균; tip인 경우에는 부모와의 차이 이용)
      - z축: 손바닥 안쪽 피부로 수직한 방향 (여기서는 global palm normal을 사용)
      - x축: 오른손 좌표계에 맞게 재계산 (x = normalize(cross(y, global_palm_normal)))
             이후 z = normalize(cross(x, y))로 정규 직교 좌표계를 구성.
    
    인자:
      joints: (21, 3) 형태의 numpy 배열 또는 리스트. 각 행은 관절의 (x, y, z) 좌표.
      axis_length: 각 축을 그릴 길이. (단위는 joints 좌표와 동일)
    """
    import numpy as np
    import plotly.graph_objects as go

    # numpy array로 변환
    joints = np.array(joints)
        # 손 메쉬 데이터 추출
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)
    x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]
    i = faces[:, 0]
    j_vals = faces[:, 1]
    k = faces[:, 2]
    

    # 정규화 함수
    def normalize(v):
        norm = np.linalg.norm(v)
        return v / norm if norm > 1e-8 else v
    
    # MANO hand skeleton hierarchy 
    # 0 (palm) -
    # 13-14-15-16 (thumb)
    # 1-2-3-17 (index)
    # 4-5-6-18 (middle)
    # 10-11-12-19 (ring)
    # 7-8-9-20 (little)

    child_dict = {
        0: [13, 1, 4, 10, 7],
        1: [2],
        2: [3],
        3: [17],
        4: [5],
        5: [6],
        6: [18],
        7: [8],
        8: [9],
        9: [20],
        10: [11],
        11: [12],
        12: [19],
        13: [14],
        14: [15],
        15: [16]
    }
    parent_dict = {
        1: 0, 2: 1, 3: 2, 4: 0,
        5: 4, 6: 5, 7: 0, 8: 7,
        9: 8, 10: 0, 11: 10, 12: 11,
        13: 0, 14: 13, 15: 14, 16: 15,
        17: 3, 18: 6, 19: 12, 20: 9
    }
    
    # 전역 palm normal 계산 (손목, 인덱스 기저, 소지 기저 이용)
    v1 = joints[1] - joints[0]    # 예: index finger base 방향
    v2 = joints[7] - joints[0]   # 예: little finger base 방향
    global_palm_normal = normalize(np.cross(v1, v2))
    # 필요에 따라 palm normal의 방향을 반전할 수 있습니다.
    # global_palm_normal = -global_palm_normal
    
    fig = go.Figure()

    # Plotly figure 생성 및 손 메쉬 추가 (투시 렌더링)
    fig.add_trace(go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j_vals, k=k,
        opacity=0.3,          # 내부가 보이도록 불투명도 낮춤
        color='lightblue',
        flatshading=True,
        name="Hand Mesh"
    ))
    
    
    # 관절 위치(마커) 표시
    fig.add_trace(go.Scatter3d(
        x=joints[:, 0],
        y=joints[:, 1],
        z=joints[:, 2],
        mode='markers',
        marker=dict(size=5, color='black'),
        name='Joints'
    ))
    
    # 각 관절마다 로컬 좌표계(축) 계산 및 표시.
    for i in range(joints.shape[0]):
        joint_pos = joints[i]
        
        # 뼈 방향 (y축) 결정:
        if i in child_dict and len(child_dict[i]) > 0:
            # 자식이 여러 개면 평균 방향 사용 (예를 들어 손목에는 여러 자식이 있음)
            directions = [joints[c] - joint_pos for c in child_dict[i]]
            y_axis = normalize(np.mean(directions, axis=0))
        elif i in parent_dict:
            # tip인 경우, 부모와의 차이 사용
            y_axis = normalize(joint_pos - joints[parent_dict[i]])
        else:
            # 특별한 경우 (예: wrist인데 자식도 없다면) 기본값 사용
            y_axis = np.array([0, 1, 0])
        
        # 로컬 x축: y축과 global palm normal의 외적
        x_axis = normalize(np.cross(y_axis, global_palm_normal))
        # 재정규화된 z축: x축과 y축의 외적 (정확한 직교 좌표계 구성)
        z_axis = normalize(np.cross(x_axis, y_axis))
        
        # 각 축 선분 그리기
        # x축: magenta
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * x_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * x_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * x_axis[2]],
            mode='lines',
            line=dict(color='magenta', width=4),
            showlegend=False
        ))
        # y축: green (뼈 방향)
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * y_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * y_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * y_axis[2]],
            mode='lines',
            line=dict(color='green', width=4),
            showlegend=False
        ))
        # z축: blue (손바닥 안쪽 방향)
        fig.add_trace(go.Scatter3d(
            x=[joint_pos[0], joint_pos[0] + axis_length * z_axis[0]],
            y=[joint_pos[1], joint_pos[1] + axis_length * z_axis[1]],
            z=[joint_pos[2], joint_pos[2] + axis_length * z_axis[2]],
            mode='lines',
            line=dict(color='blue', width=4),
            showlegend=False
        ))
    
    fig.update_layout(
        title="Hand Skeleton with Local Axes per Joint",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')
            )
        )
    )
    
    fig.show()

# 사용 예시:
# output.joints[0]가 (21,3) 형태의 핵심 hand skeleton 좌표라고 가정합니다.
display_skeleton_with_local_axes(output.joints[0], h_meshes[0], axis_length=0.01)

In [47]:
vertex_groups = {
    "Thumb1": [739, 740, 741, 755, 756, 757, 758, 759, 760, 761, 762, 763],
    "Thumb2": [31, 124, 125, 267, 698, 699, 700, 701, 704],
    "Thumb3": [7, 9, 240, 123, 126, 266],
    "Index1": [321, 322, 323, 325, 326, 328, 329, 330, 331, 342, 343, 344, 347, 349, 350],
    "Index2": [46, 47, 155, 223, 224, 237, 238, 245, 280, 281],
    "Index3": [137, 139, 140, 164, 165, 170, 171, 173, 189, 194, 195, 212],
    "Middle1": [418, 432, 433, 435, 436, 438, 439, 440, 441, 449, 454, 455, 456, 459, 461, 462],
    "Middle2": [356, 357, 372, 396, 397, 398, 402, 403],
    "Middle3": [370, 371, 374, 375, 378, 379, 380, 385, 386, 387],
    "Ring1": [523, 546, 547, 549, 550, 551, 552, 565, 566, 567, 568, 569, 570, 571, 572, 573],
    "Ring2": [468, 469, 502, 503, 506, 507, 513, 514, 516],
    "Ring3": [484, 485, 488, 489, 496, 497, 510, 579],
    "Little1": [666, 667, 668, 682, 683, 684, 685, 686, 687, 688, 689, 690, 663, 664, 669],
    "Little2": [580, 581, 598, 620, 621, 624, 625, 626, 630, 631],
    "Little3": [596, 597, 600, 601, 606, 607, 608, 613, 614, 615],
    "Palm1": [102, 278, 594, 595, 604, 605, 769, 770, 771, 775, 776],
    "Palm2": [76, 77, 141, 142, 147, 148, 196, 197, 275],
    "Palm3": [74, 75, 151, 152, 228, 268, 271, 288],
    "Palm4": [62, 63, 64, 65, 93, 132, 138, 149, 150, 168, 169],
    "Palm5": [70, 71, 72, 73, 157, 159, 188, 777],
    "Palm6": [24, 27, 66, 67, 68, 69],
    "Palm7": [32, 45, 130, 131, 243, 244, 255],
    "Palm8": [25, 109, 111, 112, 264, 265, 285],
    "Palm9": [1, 2, 4, 7, 9, 113, 115, 240]
}

In [49]:
def display_vertex_groups_on_hand_mesh(hand_mesh, vertex_groups, group_values, cmap='viridis', marker_size=5):
    """
    Visualize the hand mesh with vertex groups colored based on provided scalar values.
    
    Each vertex group (from the vertex_groups dict) is displayed using the same color,
    which is derived from a corresponding scalar value (in [0, 1]) via a specified colormap.
    
    Args:
        hand_mesh: MANO hand mesh object with `.vertices` (Nx3) and `.faces` (Mx3).
        vertex_groups: Dictionary mapping group names to lists of vertex indices.
        group_values: List or array of 24 scalar values in the range [0, 1]. The first value
                      corresponds to the first group in vertex_groups, the second to the second, etc.
        cmap (str): The Matplotlib colormap to use for mapping scalar values to colors (default: 'viridis').
        marker_size (int): Size for the markers used to display group vertices.
    """
    import numpy as np
    import plotly.graph_objects as go
    import matplotlib.colors as mcolors
    import matplotlib.pyplot as plt

    # Extract vertices and faces from the input hand mesh.
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)

    # Create a base mesh trace for the hand (semi-transparent so the groups are visible)
    base_mesh = go.Mesh3d(
        x=vertices[:, 0], 
        y=vertices[:, 1], 
        z=vertices[:, 2],
        i=faces[:, 0], 
        j=faces[:, 1], 
        k=faces[:, 2],
        opacity=0.3,
        color='lightblue',
        flatshading=True,
        name='Hand Mesh'
    )

    # Create the figure and add the mesh as a base layer.
    fig = go.Figure(data=[base_mesh])

    # Check that the number of provided values matches the number of vertex groups.
    if len(vertex_groups) != len(group_values):
        raise ValueError(f"Expected {len(vertex_groups)} group values, but got {len(group_values)}.")

    # Set up a normalization from 0 to 1 and get the colormap function.
    norm = mcolors.Normalize(vmin=0, vmax=1)
    cmap_func = plt.get_cmap(cmap)

    # Iterate over each group (using the given insertion order) and its corresponding value.
    for (group_name, indices), group_val in zip(vertex_groups.items(), group_values):
        # Map the scalar value to an RGBA color and convert it to a hexadecimal string.
        rgba = cmap_func(norm(group_val))
        color = mcolors.to_hex(rgba)

        # Extract the vertices that belong to the current group.
        group_vertices = vertices[indices]
        
        # Add a Scatter3d trace for this vertex group.
        fig.add_trace(go.Scatter3d(
            x=group_vertices[:, 0],
            y=group_vertices[:, 1],
            z=group_vertices[:, 2],
            mode='markers',
            marker=dict(size=marker_size, color=color),
            name=group_name
        ))

    # Update the layout (including camera perspective settings).
    fig.update_layout(
        title="Hand Mesh with Vertex Groups Colored",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')
            )
        )
    )
    
    fig.show()


# ----- Usage Example -----
# Suppose you already have your MANO hand mesh (for instance, from h_meshes[0]) 
# and your vertex_groups defined as in your snippet.
# Here we generate an array of 24 random scalar values (in [0, 1]) for demonstration.
import numpy as np
group_values = np.random.rand(24)

# Now display the hand mesh with the vertex groups using these values.
display_vertex_groups_on_hand_mesh(h_meshes[0], vertex_groups, group_values, marker_size=1)

In [50]:
def display_vertex_groups_faces_on_hand_mesh(hand_mesh, vertex_groups, group_values, cmap='viridis', opacity=0.7):
    """
    MANO hand mesh의 모든 face 위에, vertex 그룹에 해당하는 영역(face)을 색상 오버레이로 표시합니다.
    
    각 vertex group은 vertex 집합을 기반으로 하며, 해당 그룹에 포함된 vertex들로 이루어진 face만을 선택합니다.
    (즉, face의 모든 vertex가 해당 그룹에 속해야 합니다.)
    
    인자:
        hand_mesh: MANO hand mesh 객체. 반드시 .vertices (Nx3)와 .faces (Mx3) 속성을 가져야 합니다.
        vertex_groups: 그룹 이름을 key로, 해당 그룹에 속하는 vertex 인덱스 리스트를 value로 갖는 dict.
        group_values: 각 vertex group에 대응하는 스칼라 값 목록 (길이 24, 각 값은 0~1 범위).
        cmap (str): 스칼라 값을 색상으로 매핑할 Matplotlib colormap (기본값: 'viridis').
        opacity: 각 그룹의 face가 표시될 불투명도 (0~1, 기본값: 0.7).
    """
    import numpy as np
    import plotly.graph_objects as go
    import matplotlib.colors as mcolors
    import matplotlib.pyplot as plt

    # hand_mesh의 vertices와 faces 추출.
    vertices = np.array(hand_mesh.vertices)
    faces = np.array(hand_mesh.faces)

    # 전체 hand mesh를 반투명하게 보여주는 기본 trace 생성.
    base_mesh = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        opacity=0.3,
        color='lightblue',
        flatshading=True,
        name='Hand Mesh'
    )

    fig = go.Figure(data=[base_mesh])

    if len(vertex_groups) != len(group_values):
        raise ValueError(f"Expected {len(vertex_groups)} group values, but got {len(group_values)}.")

    # 스칼라 값을 색으로 매핑하기 위한 colormap 준비.
    norm = mcolors.Normalize(vmin=0, vmax=1)
    cmap_func = plt.get_cmap(cmap)
    
    # 각 vertex group에 대해, group 값에 따라 색상을 부여하고
    # 해당 그룹의 vertex들만 포함하는 face들을 학인하여 별도의 Mesh3d trace로 추가합니다.
    for (group_name, indices), group_val in zip(vertex_groups.items(), group_values):
        group_set = set(indices)
        # face의 모든 vertex가 해당 그룹에 속하는지 확인하여 필터링.
        group_faces = np.array([face for face in faces if set(face).issubset(group_set)])
        
        # 해당 그룹에 속하는 face가 존재할 경우, 색상 오버레이 trace를 추가.
        if group_faces.shape[0] > 0:
            rgba = cmap_func(norm(group_val))
            color = mcolors.to_hex(rgba)
            
            fig.add_trace(go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=group_faces[:, 0],
                j=group_faces[:, 1],
                k=group_faces[:, 2],
                opacity=opacity,
                color=color,
                flatshading=True,
                name=group_name
            ))
            
    fig.update_layout(
        title="Hand Mesh with Colored Vertex Group Faces",
        scene=dict(
            aspectmode='data',
            camera=dict(
                projection=dict(type='perspective')
            )
        )
    )
    
    fig.show()


# ----- Usage Example -----
# 예를 들어, MANO hand mesh (예: h_meshes[0])와 위에 정의한 vertex_groups, 그리고 24개의 스칼라 값 (각 값은 0~1)을 이용할 경우:
import numpy as np
group_values = np.random.rand(24)  # 예시용 24개 랜덤 값
display_vertex_groups_faces_on_hand_mesh(h_meshes[0], vertex_groups, group_values)